In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Create an instance of AnimalShelter with MongoDB credentials
username = "aacuser"
password = "SNHU321"
host = "nv-desktop-services.apporto.com"
port = 30517
db_name = "AAC"
collection_name = "animals"


# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, db_name, collection_name)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        html.Div([
            html.A(
                html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '250px'}),
                href='https://www.snhu.edu',  # Add the URL here
                target='_blank'  # Open the link in a new tab
            ),
            html.Div([
                html.B(html.H1('SNHU CS-340 Dashboard')),
                html.B(html.H1('Created by Noah Duarte')),
            ], style={'margin-left': '20px', 'display': 'flex', 'flex-direction': 'column', 'justify-content': 'center'}),
        ], style={'display': 'flex', 'justify-content': 'center'}),
    ]),
    html.Hr(),
    html.Div(
        #Radio Items to select the rescue filter options
        dcc.RadioItems(
            id='filter-type',
            # created the labels and keys based on the Grazioso requirements
            options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
            {'label': 'Disaster Rescue/Individual Tracking', 'value': 'Disaster Rescue/Individual Tracking'},
            {'label': 'Reset', 'value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block'},
            style={'display': 'flex', 'justify-content': 'center'}
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        #made selectable 'multi' to allow map to work with several options
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            #dcc.Graph(id='graph-id'),
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        #adjusts the read request for the desired dog type and status
    if filter_type == 'Water Rescue':
        # complex queries used to compile - was not sure which "Newfoundland" they wanted
        # so I added all that came up in animals.distinct('breed') that used "Newfoundland"
        df = pd.DataFrame(list(db.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                    {'$or': [
                                                        {'breed': 'Labrador Retriever Mix'},
                                                        {'breed': 'Chesapeake Bay Retriever'},
                                                        {'breed': 'Newfoundland'}]
                                                    },
                                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                            {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                    }]
                                        })))
    #adjusts the read request for the desired dog type and status
    elif filter_type == 'Mountain/Wilderness Rescue':
        # breeds and ages determined by Grazioso
        df = pd.DataFrame(list(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                    {'$or': [
                                                        {'breed': 'German Shepherd'},
                                                        {'breed': 'Alaskan Malamute'},
                                                        {'breed': 'Old English Sheepdog'},
                                                        {'breed': 'Siberian Husky'},
                                                        {'breed': 'Rottweiler'}]
                                                    },
                                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                            {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                    }]
                                        })))
    #adjusts the read request for the desired dog type and status
    elif filter_type == 'Disaster Rescue/Individual Tracking':
        #breeds and ages determined by Grazioso
        df = pd.DataFrame(list(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                    {'$or': [
                                                        {'breed': 'Doberman Pinscher'},
                                                        {'breed': 'German Shepherd'},
                                                        {'breed': 'Golden Retriever'},
                                                        {'breed': 'Bloodhound'},
                                                        {'breed': 'Rottweiler'}]
                                                    },
                                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                            {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                    }]
                                        })))
    #resets the search to nothing to allow all results to be displayed
    elif filter_type == 'RESET':
        df = pd.DataFrame.from_records(db.read({}))

        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
    return (data,columns)
    
    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#function to update the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # Print the structure of viewData for debugging
    print("viewData:", viewData)
    
    if viewData is None:
        print("Derived viewport data is None")
        return []  # Return an empty list if viewData is None
    
    # Create a DataFrame from the list of dictionaries
    dff = pd.DataFrame.from_records(viewData)
    
    # Print the column names of dff for debugging
    print("Column names in dff:", dff.columns.tolist())
    
    # Check if the 'breed' column is present in dff before accessing it
    if 'breed' in dff.columns:
        names = dff['breed'].value_counts().keys().tolist()
        values = dff['breed'].value_counts().tolist()
        # creates a pie chart based on the data above
        return [
            dcc.Graph(            
                figure = px.pie(
                    data_frame=dff, 
                    values = values, 
                    names = names, 
                    color_discrete_sequence=px.colors.sequential.RdBu,
                    width=800, 
                    height=500   
                )
            )
        ]
    else:
        return []  # Return an empty list if 'breed' column is not present

    
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data'),
     Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selected_rows):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    # imports the currently viewed data
    dff = pd.DataFrame.from_dict(viewData)
    markers = []
    for row_idx in selected_rows:
        marker = dl.Marker(
            position=[dff.iloc[row_idx, 13], dff.iloc[row_idx, 14]],
            children=[
                dl.Tooltip(dff.iloc[row_idx, 4]),
                dl.Popup([
                    html.H4("Animal Name"),
                    html.P(dff.iloc[row_idx, 9]),
                    html.H4("Sex"),
                    html.P(dff.iloc[row_idx, 12]),
                    html.H4("Breed"),
                    html.P(dff.iloc[row_idx, 4]),
                    html.H4("Age"),
                    html.P(dff.iloc[row_idx, 15])
                ])
            ]
        )
        markers.append(marker)

    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                *markers
            ]
        )
    ]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:10688/
viewData: None
Derived viewport data is None
viewData: [{'rec_num': 2, 'age_upon_outcome': '1 year', 'animal_id': 'A725717', 'animal_type': 'Cat', 'breed': 'Domestic Shorthair Mix', 'color': 'Silver Tabby', 'date_of_birth': '2015-05-02', 'datetime': '2016-05-06 10:49:00', 'monthyear': '2016-05-06T10:49:00', 'name': '', 'outcome_subtype': 'SCRP', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.6525984560228, 'location_long': -97.7419963476444, 'age_upon_outcome_in_weeks': 52.9215277777778}, {'rec_num': 1, 'age_upon_outcome': '3 years', 'animal_id': 'A746874', 'animal_type': 'Cat', 'breed': 'Domestic Shorthair Mix', 'color': 'Black/White', 'date_of_birth': '2014-04-10', 'datetime': '2017-04-11 09:00:00', 'monthyear': '2017-04-11T09:00:00', 'name': '', 'outcome_subtype': 'SCRP', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.5066578739455, 'location_long': -97.3408780722188